# Bayesian Optimization for Truss Structures

Group: TRUSS 01

Team members: 
- Albrecht, Franziska
- Bakker, Christiaan
- Scheurwater, Robbin
- Skorobogatova, Ekaterina

## Introduction

![37-bar truss structure](./Images/Truss_structure.jpeg)
Figure 1: 37-bar truss structure

Truss optimization is important because it enables the design of lightweight, cost-effective, and environmentally sustainable structures. However, evaluating truss structures, which are typically multi-dimensional optimization problems, can be computationally expensive. Compared to meta-heuristic optimization algorithms, the Bayesian optimizer is more data-efficient, as it can incorporate prior knowledge about the function being optimized. To investigate the performance of the Bayesian optimizer for problems with different dimensions, it is implemented and analysed in this project on a truss structure, where the dimensionality is increased step by step [1, 2]

The aim of this project is to find the optimum design for the 37-bar truss structure shown in Figure 1. The objective is to minimize the total weight of the truss structure and at the same time meet the requirements for the lowest natural frequencies.

<u>Definition of the optimization problem:</u>

Objective function: $$min  R(x)=\sum_{q=1}^{37}A_q \cdot \rho_q \cdot L_q$$

where
- R(x): weight of the truss structure
- $A_q $: cross section area
- $ \rho_q$: density
- $L_q$: length of the qth truss member

Such that:  $$g_1(x)=20-w_1 \leq 0$$
            $$g_2(x)=40-w_2 \leq 0$$
            $$g_3(x)=60-w_3 \leq 0$$

To evaluate the given optimization problem using a Bayesian optimizer, the objective of weight minimization is combined with the constraints on the lowest natural frequencies. For this purpose, the following objective function is formulated based on the augmented Lagrange function:
$$L(x, p) = \sum_{q=1}^{37}A_q \cdot \rho_q \cdot L_q + p \cdot \sum_{j=1}^{3} [max(0,g_j(x))]^2$$

where $p :$ penalty weight


## Import libraries and auxiliary files

<u>Python libraries and modules:</u>

In [ ]:
sys.path.append('../pyJive/')
sys.path = list(set(sys.path))

import torch
from torch.distributions import Normal
import pandas as pd

import math
import numpy as np
import sys

from utils import proputils as pu
import main
from names import GlobNames as gn
from utils import nodeset as ns
import pandas as pd
import torch
import random
from objective_utils import *
import matplotlib.pyplot as plt
import cProfile

<u>Objective function</u>:

The complexity of the given optimization problem is built up step by step:

| Step | Dimensionality | Design variables |
| :-:  | :-:            | :-               |
| 1    | 1D             | y-coordinate of all top nodes |
| 2    | 2D             | y-coordinate of all top nodes, single A for all members |
| 3    | 6D             | independent y-coordinates of all top nodes |
| 4    | 19D            | independent y-coordinates of all top nodes, independent A for all members |

For each step of dimensionality a single objective function is created in the cell below. 

The implementation of the objective functions can be found in the file: `objective_utilis.py`. 


In [ ]:
def objective_function_1D(input_array, constraint_weight):
    assert len(input_array) == 1, f"Input array must be one-dimensional, but got shape {input_array.shape}."

    y = input_array[0]

    return oneD_lagrange_optimization(y, constraint_weight=constraint_weight, df=df, props=props, globdat=globdat)

def objective_function_2D(input_array, constraint_weight):
    assert len(input_array) == 2, f"Input array must be two-dimensional, but got shape {input_array.shape}."

    y = input_array[0]
    A = input_array[1]

    return twoD_lagrange_optimization(y, A, df=df, props=props, globdat=globdat, constraint_weight=constraint_weight)

def objective_function_6D(input_array, constraint_weight):
    assert len(input_array) == 6, f"Input array must be six-dimensional, but got shape {input_array.shape}."

    y_values = input_array[:5]
    A = input_array[5]

    return sixD_lagrange_optimization(y_values, A, constraint_weight=constraint_weight, df=df, globdat=globdat, props=props)

def objective_function_19D(input_array, constraint_weight):
    assert len(input_array) == 19, f"Input array must be nineteen-dimensional, but got shape {input_array.shpe}."

    y_values = input_array[:5]
    A_values = input_array[5:]

    return nineteenD_lagrange_optimization(y_values, A_values, constraint_weight, df=df, props=props, globdat=globdat)

## Bayesian Optimizer

Bayesian Optimization (BO) is a machine-learning-based optimization technique that is particularly well-suited for problems where evaluating the objective function is computationally expensive. Unlike conventional optimization methods that require a large number of function evaluations, BO efficiently explores the design space by leveraging a probabilistic surrogate model, a Gaussian Process (GP), to approximate the objective function.

In the context of truss structure optimization, evaluating a design involves solving a structural mechanics problem, often using finite element analysis, which can be computationally costly. Standard optimization approaches, such as gradient-based or evolutionary algorithms, may require thousands of evaluations to converge to an optimal design. BO mitigates this issue by strategically selecting the most informative points to evaluate, balancing exploration (searching unexplored regions) and exploitation (refining promising designs).

The optimization problem addressed in this project involves minimizing the weight of a truss structure while satisfying constraints on its first three natural frequencies. The design variables include the y-coordinates of the top nodes and the cross-sectional areas of the truss members, as stated above. The Bayesian optimization framework is employed to efficiently navigate this high-dimensional design space, ensuring that structural performance requirements are met while minimizing computational effort.

The BO process consists of the following key components:

1. **Surrogate Model:** A Gaussian Process is used to approximate the objective function, providing both predictions and uncertainty estimates.
2. **Acquisition Function:** This function determines the next point to evaluate by balancing exploration and exploitation.
3. **Evaluation & Update:** The selected design is evaluated using a finite element solver and the surrogate model is updated with the new data.
4. **Iteration & Convergence:** The process is repeated until a stopping criterion is met, in this case achieving sufficient convergence based on the acquisition function.

This framework will be tested on truss structure problems of increasing complexity, from 1D optimization to a full 19D problem where all cross-sectional areas and nodal positions are independently optimized. The effectiveness of Bayesian Optimization will be analyzed in terms of computational efficiency and solution quality compared to traditional optimization methods.


## BayesianOptimizer Class Documentation

To structure the Bayesian optimization process, a class named `BayesianOptimizer` was implemented. This class consists of 10 functions, each contributing to different steps of the optimization. Figure 2 presents a flowchart illustrating the structure, based on [3]. The class and its functions are shown and explained below.

![Flowchart of the Bayesian Optimizer](./Images/Bayesian_workflow.jpg)

Figure 2: Flowchart of the Bayesian OptimizerThe flowchart in Figure 2 shows the structure of the implemented Bayesian optimizer based on [3].

### Class Definition

```python
class BayesianOptimizer:
    def __init__(dimension, bounds, kernel, hyperparams, objective_function, SEED, device=device)
```

### Class Attributes

- `dimension (int)`: The dimensionality of the optimization problem.
- `bounds (torch.Tensor)`: The optimization bounds for each dimension (min, max).
- `kernel (function)`: The kernel function for the Gaussian Process.
- `hyperparams (dict)`: Hyperparameters for the kernel and optimization (beta, noise, etc.).
- `objective_function (function)`: The objective function to be optimized.
- `SEED (int)`: The integer sets the seed for the optimizer
- `device (torch.device)`: The computational device (cuda or cpu).
- `X (torch.Tensor):` The input data points (shape: n x d).
- `y (torch.Tensor)`: The observed function values for X.
- `y_norm ($torch.Tensor)`: The normalized y values.
- `global_y_min (float)`: The global minimum value of y.
- `global_y_max (float)`: The global maximum value of y.
- `log_df (pd.DataFrame)`: Logs of optimization progress (iterations, best values, hyperparameters).

### Methods

#### 1. \_\_init\_\_

```python
def __init__(self, dimension, bounds, kernel, hyperparams, objective_function, SEED, device=device):
```

The `__init__` method initializes the optimizer by setting up the dimensions, bounds, kernel, and hyperparameters for the optimization process. It establishes a controlled random environment using the provided seed value to ensure reproducibility. Key parameters such as `hit` (initialized to zero) and `hit_tolerance` (set to `5e-10`) are defined for later use in the optimization logic.

Additionally, several attributes are initialized but left undefined at this stage, including:

- `X` and `y`: Placeholder tensors for the input data points and their corresponding function values.
- `y_norm`: A normalized version of `y` for consistent scaling.
- `global_y_min` and `global_y_max`: Values used for normalization, representing the minimum and maximum of `y`.
- `alpha` and `L`: Variables reserved for use in the Cholesky decomposition, which will be employed in subsequent calculations.

The method also initializes the DataFrame `log_df` to log the progress of the optimization. The columns of this DataFrame are:

- **Iteration**: The current iteration number.
- **Best_X**: The new input based on the prior, which is later added to the posterior and becomes the new prior.
- **Iteration_Y**: The output of the objective function based on `Best_X`.
- **Best_Y**: The best output of the objective function found across all iterations.
- **Best_EI**: The Expected Improvement (EI) based on `Best_X`.
- **Sig_f**: The kernel scale parameter of the Gaussian Process (GP) used to find `Best_X` with the expected improvement.
- **Length**: The kernel length scale parameter of the GP used to find `Best_X`.



#### 2. initialize

```python
def initialize(self, n_initial_points):
```
The `initialize` function samples `n_initial_points` random points within the defined bounds and computes their corresponding function values.

- `self.X` generates `n_initial_points` random samples within the specified bounds.  
  - The bounds for both the length and the cross-section are from 1 to 5, measured in meters (m) and square centimeters (cm²), respectively.  
- The `objective_function` is used to compute the output for each sampled input.
- The global minimum and maximum values of the objective function output are determined:
  - `self.global_y_min`: Stores the lowest observed output value.
  - `self.global_y_max`: Stores the highest observed output value.
- `self.y_norm` is then computed as a normalized version of `self.y`, ensuring a consistent scale for optimization.The `initialize` function samples `n_initial_points` random points within the defined bounds and computes their corresponding function values.
---

#### 3. update\_hyperparameters

```python
def update_hyperparameters(self):
```

The `update_hyperparameters` function optimizes the kernel hyperparameters `sig_f` (signal variance) and `length` (length scale) by maximizing the log marginal likelihood. This function is only executed when explicitly called in `optimize`.

  
The Adam optimizer is chosen for updating the hyperparameters because it is well-suited for non-convex optimization problems and avoids getting stuck in poor local minima. Adam uses adaptive learning rates based on first and second moments of gradients, making it efficient for Bayesian Optimization where updates need to be smooth and stable.

#### How it works  
1. The hyperparameters `sig_f` and `length` are initialized as a `torch.tensor` with `requires_grad=True` so they can be optimized.
2. The Adam optimizer is initialized with a learning rate of 0.01 to balance convergence speed and stability. Altough the Adam optimizer changes the learning rate automatically, thus the initially defined value is arbitrary.
3. A loop runs for 20 iterations, where:
   - The optimizer gradients are reset (`zero_grad`).
   - The hyperparameter values are clamped between 1e-3 and 10.0 to ensure numerical stability.
   - The covariance matrix `k11` is computed using the kernel function with the current `sig_f` and `length` values.
   - A small noise term (`noise² + 1e-5`) is added to the diagonal of `k11` to improve numerical stability.
   - The Cholesky decomposition is used to compute the inverse of `k11`, and the log marginal likelihood is calculated.
   - The negative log marginal likelihood (loss) is minimized to find the optimal `sig_f` and `length` values.
4. After 20 iterations, the optimized values of `sig_f` and `length` are stored in `self.hyperparams`.

These updates allow the optimizer to adaptively refine the kernel parameters, improving the Gaussian Process model for more accurate predictions and better-informed search for the minimum.

---

#### 4. update\_normalization

```python
def update_normalization(self):
```

The `update_normalization` function ensures that the objective function outputs remain within a consistent numerical range by normalizing `y`. This is important for stability and efficiency in Bayesian Optimization.

The ouput `y` (the objective function outputs) is normalised as the values  can vary significantly depending on the problem. As `y` has a very large range because of the constraint weight used in the objective function, it can lead to numerical instability and poor scaling in GP regression. Normalizing `y` brings all values into a controlled range, improving the performance of the optimization process.

How it works: 
1. The global minimum and maximum values of `y` are updated:
   - `self.global_y_min`: Stores the lowest observed function value.
   - `self.global_y_max`: Stores the highest observed function value.
2. The normalized values `y_norm` are computed using the formula for min-max normalisation.
   The small constant `1e-12` is added to avoid division by zero when `global_y_max` and `global_y_min` are close.

Since Expected Improvement (EI) depends on the predicted mean and variance of the Gaussian Process, normalizing `y` ensures that all function values are treated on the same scale, preventing larger values from dominating the search. This helps the acquisition function balance exploration and exploitation more effectively.

By keeping `y` in a stable range, the optimizer avoids numerical instabilities and improves the efficiency of EI-based point selection, leading to better optimization performance.

---

#### 5. compute\_posterior

```python
def compute_posterior(self, X_hat):
```

The `compute_posterior` function computes the posterior mean and covariance for a given set of candidate points using GP regression.

How it works:
1. The input `X_hat` (candidate points) is first moved to the correct device (`cpu` or `cuda`).
2. The kernel function is used to compute:
   - `k12`: The covariance between the observed points `X` and the candidate points `X_hat`.
   - `k22`: The covariance between candidate points themselves.
3. The posterior mean is calculated as:
   $
   \mu_{\text{norm}} = k_{12}^T \alpha
   $
   where `alpha` is precomputed in the Cholesky decomposition step in the `cholesky` function.
4. The posterior covariance is computed using:
   $
   \text{cov}_{\text{norm}} = k_{22} - k_{12}^T v
   $
   where `v` is obtained from solving a linear system using Cholesky decomposition.

Gaussian Process regression requires solving a system involving the covariance matrix, which can be large and difficult to invert. Instead of computing the inverse directly, which can be unstable and inefficient, Cholesky decomposition is used in the `cholesky` function because:

- More numerically stable: Direct matrix inversion can introduce large numerical errors. Cholesky decomposition provides a stable factorization of the covariance matrix.
- Computational efficiency: Cholesky decomposition has a complexity of O(n³), which is the same as direct inversion but avoids unnecessary computations.
- Ensures positive definiteness: The covariance matrix must be positive definite. If it is nearly singular, adding a small jitter term (`1e-6` to `1e-2`) in the `cholesky` function ensures stability.

---

#### 6. cholesky

```python
def cholesky(self):
```

The `cholesky` function is responsible for computing the Cholesky decomposition of the covariance matrix, which is essential for GP regression. It provides a stable and efficient way to solve the linear system required for making predictions.

How is works: 
1. The covariance matrix `k11` is computed using the kernel function.
2. A small jitter term (`1e-6`) is added to the diagonal to ensure numerical stability.
3. The function attempts to compute the Cholesky decomposition:
   - If decomposition fails due to numerical issues, the jitter is increased iteratively (`jitter_increment = 10`).
   - This process continues until a maximum jitter (`1e-2`) is reached.
   - If decomposition still fails, an error is raised, indicating that the matrix is not positive definite.
   - This addition was used as when optimising the hyperparameters for the RBF kernel, the cholesky matrixes often weren't positive definite. By adding increased jitter this problem was resolved. For the other three kernels this problem did not arise.
4. The decomposition produces a lower triangular matrix `L` such that:
   $
   K_{11} = L L^T
   $
5. The weights `alpha` are computed using:
   $
   \alpha = K_{11}^{-1} y_{\text{norm}}
   $
   Instead of computing `K_{11}^{-1}` directly, which is numerically unstable, `alpha` is obtained efficiently using:
   ```python
   alpha = torch.cholesky_solve(self.y_norm.unsqueeze(-1), L)
   ```
6. The function returns $\alpha$ and $L$, which are used in the `compute_posterior` function stated above.

---

#### 7. expected\_improvement

```python
def expected_improvement(self, X_hat):
```
Expected Improvement (EI) is used as the acquisition function because it balances exploration and exploitation . It encourages selecting points that are likely to improve upon the current best observed valu* while avoiding unnecessary sampling in well-explored areas. Compared to alternatives like Upper Confidence Bound (UCB), EI produces smoother acquisition surfaces, leading to more stable optimization.

How it works  :
1. Retrieves the posterior mean and variance from `compute_posterior`.  
2. Computes potential improvement over the best observed function value (`y_min`).  
3. Assigns higher EI values to points with low function values or high uncertainty.  
4. Returns EI values, which are used to suggest the next sample point.  

---

#### 8. suggest_next_point

```python
def suggest_next_point(self):
```

How it works:  
1. Randomly sample candidate points within the search space.  
   - A set number of points (`X_candidates`) is randomly drawn.  
   - These serve as initial guesses for the next best point.  

2. Optimize each candidate using Adam to maximize Expected Improvement (EI).  
   - The EI function is negated (since PyTorch optimizers minimize by default).  
   - Adam is used instead of standard gradient descent because:  
     - It adapts the learning rate based on past gradients, improving stability.  
     - It prevents overshooting and converges faster than standard methods.  

3. Tolerance check prevents over-optimization.  
   - If EI stops improving significantly over iterations, optimization terminates early.  
   - A tolerance threshold is set using `self.tol * self.beta`, avoiding unnecessary fine-tuning.  

4. Select the candidate with the highest EI.  
   - After all candidates are optimized, the one with the best EI is chosen as the next sample point.  
   - This ensures that the optimizer moves towards areas with high potential for improvement.  

The function returns the best candidate point for the next function evaluation along with its corresponding EI value.  


---

#### 9. observe_new_points

```python
def observe_new_points(self, X\_new, y\_new):
```

How it works:
1. Adds the new observed point to the prior
    - The newly suggested input (`X_new`) and its corresponding function output (`y_new`) are added to `self.X` and `self.y`.
2. Recomputes normalization of outputs
    - The global minimum and maximum of y are updated and the ouputs are renormalized.

---

#### 10. optimize

```python
def optimize(self, n\_iterations, depth=1e-24, update=False, plot=False):
```

The `optimize` function is the main loop of Bayesian Optimization. It iteratively selects new points, evaluates the objective function, updates the Gaussian Process and logs the results until convergence.

---

How it Works – Step-by-Step Process  

1. Initialize Cholesky decomposition (`cholesky`)  
   - At the start, Cholesky decomposition of the covariance matrix is computed.  
   - This ensures that the Gaussian Process is correctly initialized before selecting new points.  

2. Loop over iterations (`n_iterations`)  
   Each iteration follows this sequence:  

   a. Compute Expected Improvement (`expected_improvement`)  
   - The optimizer determines which input values would provide the highest improvement over the best known function value.  
   - This is done using the Expected Improvement (EI) function, which considers both mean prediction and uncertainty.  

   b. Suggest a new input (`suggest_next_point`)  
   - A set of random candidate points is generated.  
   - The EI values of these points are optimized using Adam, and the candidate with the highest EI is selected.  

   c. Evaluate the objective function 
   - The objective function is evaluated at the newly selected input.  
   - The corresponding output (`y_new_val`) is obtained.  

   d. Observe the new point (`observe_new_points`)  
   - The new input and output are added to the dataset.  
   - The global min and max values are updated, and the function outputs are re-normalized.  

   e. Recompute Cholesky decomposition (`cholesky`)  
   - Since the dataset has changed, the Cholesky decomposition is updated to ensure the Gaussian Process predictions remain accurate.  

   f. Update Hyperparameters (`update_hyperparameters`) 
   - Hyperparameter updates (`sig_f`, `length`) are done at specific iterations.  
   - The exact update schedule was determined through testing using a fixed seed, ensuring all other factors remained constant.  
   - This allowed tuning of the frequency and conditions under which hyperparameters were updated to maximize performance.  
   - For 1D: Every 5 iterations.  
   - For 2D, 6D, 19D: Every 10 or 50 iterations, depending on the phase of optimization.  

   g. Lowering of `beta` (Exploration-Exploitation Balance)  
   - `beta` is a scaloring factor that determines the uncertainty of the GP by scaling the standard devation.
   - `beta` is reduced periodically to shift from exploration (early iterations) to exploitation (later iterations).  
   - The iteration at which `beta` is reduced was also determined by testing with the same seed, ensuring an optimized schedule.  
   - For 1D & 2D: Every 20 iterations.  
   - For 6D & 19D: Every 50 iterations after 500 iterations.  

   h. Log the iteration in the DataFrame (`df`)  
   - The following values are recorded:  
     - Iteration number 
     - Best X: The newly chosen input  
     - Iteration Y: The corresponding function output  
     - Best Y: The best function value found so far  
     - Best EI: The EI value at the new point  
     - Sig_f and Length: The kernel parameters used in the Gaussian Process  

3. Check Stopping Condition (`depth` and `beta`)
   - If EI falls below (`depth * self.beta`) for five consecutive iterations, optimization stops.  
   - This prevents unnecessary computations when no significant improvement is expected.
   - The depth was chosen by testing different options. In the 1D and 2D case this was easy, by checking how long it took for the optimiser to reach the lowest value and after that continue with very similar points we could decrease the depth. The depth for the 6D and 19D was chosen based on the outcomes of different runs.

4. Plot Results for 1D and 2D Cases  
   - If `plot=True`, visualizations are generated:  
     - 1D case: The posterior mean, uncertainty, and EI are plotted.  
     - 2D case: Contour plots of the posterior and EI show how the optimizer explores the search space.
   - This was used to check if the optimizer worked as intended and the highest EI was indeed chosen.

---
**Summary of the Function Flow**  

1. Compute Cholesky decomposition(once at the start).  
2. For each iteration:  
   - Compute **EI**.  
   - Suggest the **next input**.  
   - Evaluate the **objective function**.  
   - Observe the new point (**update dataset & normalization**).  
   - Recompute **Cholesky decomposition**.  
   - **Update hyperparameters** at scheduled iterations.  
   - **Reduce `beta`** at predefined steps.  
   - Log results in the **DataFrame**.  
   - Stop if **EI remains too low** for multiple iterations.  
3. If `plot=True`, generate **visualizations** for 1D and 2D cases.  

---






In [1]:
class BayesianOptimizer:
    def __init__(self, dimension, bounds, kernel, hyperparams, objective_function, SEED, device=device):
        self.dimension = dimension
        self.bounds = torch.tensor(bounds, dtype=torch.float32, device=device).repeat(dimension, 1) #In form of (min, max)
        self.kernel = kernel
        self.hyperparams = hyperparams
        self.objective_function = objective_function
        self.beta = hyperparams['beta']
        self.constraint_weight = hyperparams['constraint_weight']
        self.noise = hyperparams['noise']
        self.n_suggestions = hyperparams['n_suggestions']
        self.adam_iteration = hyperparams['adam_iteration']
        self.device = device
        self.hit = 0
        self.tol = 5e-10
        self.hit_tol = 0

        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(SEED)
            torch.cuda.manual_seed_all(SEED)

        
        self.X = None            # shape (n, d)
        self.y = None            # shape (n,)
        self.y_norm = None       # shape (n,) - normalized
        self.global_y_min = None
        self.global_y_max = None
        self.alpha = None        
        self.L = None            
        
        
        self.log_df = pd.DataFrame(columns=["Iteration", "Best_X", "Iteration_Y", "Best_Y", "Best_EI", "Sig_f", "Length", "Beta"])
        

    def initialize(self, n_initial_points):
        """Initialize with random points in the given 1D bounds, plus their objective values."""
        print(f'Initializing with {n_initial_points} initial points')

        original_stdout = sys.stdout
        sys.stdout = open('/dev/null', 'w')
        
        self.X = torch.rand((n_initial_points, dimension), device=self.device, dtype=torch.float32)
        self.X = self.X * (self.bounds[0, 1] - self.bounds[0, 0]) + self.bounds[0, 0]

        self.y = torch.tensor(
            [self.objective_function(x.cpu().numpy(), self.constraint_weight) for x in self.X],
            device=self.device, dtype=torch.float32
        )
        
        sys.stdout.close()
        sys.stdout = original_stdout

        self.global_y_min = self.y.min().item()
        self.global_y_max = self.y.max().item()

        self.y_norm = (self.y - self.global_y_min) / (self.global_y_max - self.global_y_min + 1e-12)

    def update_hyperparameters(self):
        """Optimize hyperparameters `sig_f` and `length` periodically."""
        hyperparam_values = torch.tensor([
            self.hyperparams['sig_f'],
            self.hyperparams['length']
        ], requires_grad=True, device=self.device)
    
        optimizer = torch.optim.Adam([hyperparam_values], lr=0.01)
    
        for _ in range(20): 
            optimizer.zero_grad()

            hyperparam_values.data = torch.clamp(hyperparam_values, min=1e-3, max=10.0) #Added for 2D error
    
            noise = self.hyperparams['noise']
            k11 = self.kernel(self.X, self.X, {
                'sig_f': hyperparam_values[0],
                'length': hyperparam_values[1]
            })
            k11 += (noise**2 + 1e-5) * torch.eye(self.X.size(0), device=self.device, dtype=torch.float32)
    
            L = torch.linalg.cholesky(k11)
            alpha = torch.cholesky_solve(self.y_norm.unsqueeze(-1), L)
    
            log_marginal_likelihood = -0.5 * (self.y_norm.unsqueeze(-1).T @ alpha).squeeze()
            log_marginal_likelihood -= torch.sum(torch.log(torch.diag(L)))
            log_marginal_likelihood -= 0.5 * self.X.size(0) * torch.log(torch.tensor(2 * torch.pi, device=self.device))
    
            loss = -log_marginal_likelihood
            loss.backward()
            optimizer.step()
    
        self.hyperparams['sig_f'] = hyperparam_values[0].item()
        self.hyperparams['length'] = hyperparam_values[1].item()


    def update_normalization(self):
        """Recompute global_y_min, global_y_max, and y_norm after each new observation."""
        self.global_y_min = float(self.y.min().item())
        self.global_y_max = float(self.y.max().item())
        denom = (self.global_y_max - self.global_y_min) + 1e-12
        self.y_norm = (self.y - self.global_y_min) / denom

    def compute_posterior(self, X_hat):
        """
        Compute posterior *in normalized y-space*.
        Returns mu_norm, cov_norm (both are in normalized scale).
        """
        
        X_hat = X_hat.to(self.device)
        k12 = self.kernel(self.X, X_hat, self.hyperparams).float()
        k22 = self.kernel(X_hat, X_hat, self.hyperparams).float()

        mu_norm = (k12.T @ self.alpha).squeeze(-1)
        v = torch.cholesky_solve(k12, self.L)
        cov_norm = k22 - k12.T @ v

        return mu_norm, cov_norm
    
    def cholesky(self):
        """
        Cholesky decomposition based on X. Returns alpha
        """
        jitter = 1e-6  
        max_jitter = 1e-2  
        jitter_increment = 10  

        k11 = self.kernel(self.X, self.X, self.hyperparams).float()
        k11 += self.noise**2 * torch.eye(self.X.size(0), device=self.device, dtype=torch.float32)

        while jitter <= max_jitter:
            try:
                k11_with_jitter = k11 + jitter * torch.eye(self.X.size(0), device=self.device, dtype=torch.float32)
                L = torch.linalg.cholesky(k11_with_jitter)
                break  
            except torch._C._LinAlgError:
                jitter *= jitter_increment  
        else:
            raise RuntimeError("Cholesky decomposition failed. Matrix is not positive definite, even with added jitter.")
                  
        alpha = torch.cholesky_solve(self.y_norm.unsqueeze(-1), L)
        
        return alpha, L
        

    def expected_improvement(self, X_hat):
        """
        EI in normalized scale:
         - If we are "minimizing" the raw objective, we consider the best (lowest) y_norm so far.
         - The formula uses y_min_norm, mu_norm, sigma_norm, etc.
        """
        
        X_hat = X_hat.to(self.device)
        mu_norm, cov_norm = self.compute_posterior(X_hat)
        sigma_norm = torch.sqrt(torch.clamp(torch.diag(cov_norm), min=1e-12))

        # y_norm_min = best (lowest) point in normalized scale
        y_norm_min = self.y_norm.min().item()

        normal = Normal(0, 1)

        u = (y_norm_min - mu_norm) / (sigma_norm * self.beta + 1e-12)

        u = torch.where(torch.isnan(u), torch.tensor(0.0, device=self.device), u)
        u = torch.clamp(u, min=-10.0, max=10.0)

        EI = (sigma_norm * self.beta) * (u * normal.cdf(u) + normal.log_prob(u).exp())
        return EI

    def suggest_next_point(self):
        """
        Randomly initialize candidate points, then do local gradient-based
        optimization (minimize -EI) to find maxima of EI in normalized scale.
        """
        X_candidates = torch.rand((self.n_suggestions, self.dimension), device=self.device)
        X_candidates = self.bounds[:, 0] + X_candidates * (self.bounds[:, 1] - self.bounds[:, 0])
        
        best_EI = float('-inf')
        x_proposed = None

        for x_i in X_candidates:
            x_i = x_i.unsqueeze(0).clone().detach().requires_grad_(True)
            optimizer = torch.optim.Adam([x_i], lr=0.01) #learningrate can be changed?

            for _ in range(self.adam_iteration):
                optimizer.zero_grad()
                # Minimize negative EI => maximize EI
                neg_ei = -self.expected_improvement(x_i)
                neg_ei.backward()
                optimizer.step()

                x_i.data = torch.max(x_i.data, self.bounds[:, 0])
                x_i.data = torch.min(x_i.data, self.bounds[:, 1])

                current_EI = -neg_ei.item()

                if abs(current_EI - best_EI) < (self.tol * self.beta):

                    if _ == 0:
                        self.hit_tol += 1
                        if self.hit_tol == 5:
                            self.tol = self.tol * 0.1
                            # print(f"tol lowered: {self.tol}")
                            self.hit_tol = 0
                    else:
                        self.hit_tol = 0

                    break

            with torch.no_grad():
                current_EI = self.expected_improvement(x_i).item()
                if current_EI > best_EI:
                    best_EI = current_EI
                    x_proposed = x_i.detach()

        if x_proposed is not None:
            x_proposed = x_proposed.reshape(1, -1)

        return x_proposed, best_EI

    def observe_new_points(self, X_new, y_new):
        """
        Add the newly observed point(s) to self.X and self.y, then re-normalize.
        """
        
        X_new = X_new.to(self.device)
        y_new = y_new.to(self.device)
        
        self.X = torch.cat((self.X, X_new), dim=0)
        self.y = torch.cat((self.y, y_new), dim=0)

        self.update_normalization()

    
    def optimize(self, n_iterations, depth=1e-24, update=False, plot=False):
        """
        Main optimization loop. Plots the GP posterior and EI each iteration for 1D and 2D if True. Updates hyperparameters if true.
        """
        
        start_time = time.time()
        y_min = 1e8

        if self.dimension == 1 and plot:
            output_folder = "plot_1D"
            os.makedirs(output_folder, exist_ok=True)
            x_candidates = torch.linspace(self.bounds[0, 0], self.bounds[0, 1], 100, dtype=torch.float32).unsqueeze(1).to(self.device)
            plt.figure(figsize=(12, 4 * n_iterations))
        
        elif self.dimension == 2 and plot:
            output_folder_2 = "plot_2D"
            os.makedirs(output_folder_2, exist_ok=True)
            x1_candidates = torch.linspace(self.bounds[0, 0], self.bounds[0, 1], 100, dtype=torch.float32)  # Range for the first dimension
            x2_candidates = torch.linspace(self.bounds[0, 0], self.bounds[0, 1], 100, dtype=torch.float32)  # Range for the second dimension
            x_candidates = torch.cartesian_prod(x1_candidates, x2_candidates).to(self.device)

            max_rows = (n_iterations + 9) // 10
            fig, axes = plt.subplots(max_rows, 2, figsize=(12, 4 * max_rows), squeeze=False)
            row_index = 0

        self.alpha, self.L = self.cholesky()

        for i in range(1, n_iterations + 1):
            if i % 50 == 0:
                print(f'Iteration {i}')
                
            if self.dimension == 1:
                depth = 1e-4 
            elif self.dimension == 2:
                depth = 1e-15
            elif self.dimension in [6, 19]:
                depth = 1e-30

            if i % 20 == 0 and self.dimension in [1, 2]:
                self.beta = self.beta * 0.75   
            elif i % 50 == 0 and i > 500 and self.dimension in [6, 19]:
                self.beta = self.beta * 0.75
                
            #Updating hyperparameters wrt number of iteration and dimension
            if update:
                if self.dimension == 1 and i % 5 == 0:
                    self.update_hyperparameters()

                if self.dimension in [2, 6, 19] and (i % 10 == 0):
                    if self.dimension == 2 and (i < 100):
                        self.update_hyperparameters()
                    elif self.dimension in [6, 19] and (i < 150):
                        self.update_hyperparameters()

                if self.dimension == 2 and (i > 100) and (i % 50 == 0):
                    self.update_hyperparameters()

                if self.dimension in [6, 19] and (i > 150) and (i < 500) and (i % 50 == 0):
                    self.update_hyperparameters()

            x_new, best_ei = self.suggest_next_point()
            
            # Suppress printing pyjive output
            original_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')

            y_new_val = self.objective_function(x_new.cpu().numpy().flatten(), self.constraint_weight)
            y_new = torch.tensor([y_new_val], device=self.device, dtype=torch.float32)

            sys.stdout.close()
            sys.stdout = original_stdout

            self.observe_new_points(x_new, y_new)
            self.alpha, self.L = self.cholesky()

            #Saving values to a log
            if y_new_val < y_min:
                y_min = y_new_val
            
            #When to stop iterating
            if best_ei < depth * self.beta:
                self.hit += 1
                if self.hit >= 5:
                    print(f"Best EI: {best_ei} Hit")
                    break
            else:
                self.hit = 0
            
            self.log_df = pd.concat([
                self.log_df,
                pd.DataFrame({
                    "Iteration": [i],
                    "Best_X": [x_new.cpu().numpy().flatten().tolist()],
                    "Iteration_Y": [y_new_val],
                    "Best_Y": [y_min],
                    "Best_EI": [best_ei],
                    "Sig_f": self.hyperparams['sig_f'],
                    "Length": self.hyperparams['length'],
                    "Beta": self.beta
                })
            ], ignore_index=True)
            
            #Plotting design variables
            TITLE_FONT_SIZE = 20
            AXIS_LABEL_FONT_SIZE = 18
            TICK_FONT_SIZE = 14
            LINE_WIDTH = 3
            FIGURE_SIZE = (12, 6)
            
            if plot:
                if self.dimension == 1:
                    mu_norm, cov_norm = self.compute_posterior(x_candidates)
                    std_norm = torch.sqrt(torch.diag(cov_norm))

                    y_range = (self.global_y_max - self.global_y_min) + 1e-12
                    mu_rescaled = mu_norm * y_range + self.global_y_min
                    std_rescaled = std_norm * y_range

                    ei_vals = self.expected_improvement(x_candidates).cpu().numpy()
                    ei_vals_transformed = np.maximum(ei_vals, 1e-6)

                    # LEFT: GP Posterior
                    plt.figure(figsize=FIGURE_SIZE)
                    plt.subplot(1, 2, 1)
                    plt.title(f"Iteration {i} - GP Posterior", fontsize=TITLE_FONT_SIZE)
                    plt.plot(x_candidates.cpu().numpy(), mu_rescaled.cpu().numpy(),
                             'b-', label="GP Mean", linewidth=LINE_WIDTH)
                    plt.fill_between(
                        x_candidates.cpu().numpy().flatten(),
                        (mu_rescaled - 1.96 * std_rescaled).cpu().numpy(),
                        (mu_rescaled + 1.96 * std_rescaled).cpu().numpy(),
                        color='blue', alpha=0.3, label="GP Confidence"
                    )
                    plt.scatter(
                        self.X.cpu().numpy().flatten(),
                        self.y.cpu().numpy(),
                        color="red", zorder=5, label="Evaluated Points"
                    )
                    plt.axvline(x_new.item(), color="gray", linestyle="--", label="Next Point", linewidth=LINE_WIDTH)
                    plt.xlabel("Height", fontsize=AXIS_LABEL_FONT_SIZE)
                    plt.ylabel("Objective output", fontsize=AXIS_LABEL_FONT_SIZE)
                    plt.ylim(0, 110000)
                    plt.xticks(fontsize=TICK_FONT_SIZE)
                    plt.yticks(fontsize=TICK_FONT_SIZE)
                    plt.legend()

                    # RIGHT: EI
                    plt.subplot(1, 2, 2)
                    plt.title(f"Iteration {i} - EI | Max: {best_ei:.4f}")
                    plt.plot(x_candidates.cpu().numpy(), ei_vals_transformed, 'orange', label="EI", linewidth=LINE_WIDTH)
                    plt.axvline(x_new.item(), color="gray", linestyle="--", label="Next Point", linewidth=LINE_WIDTH)
                    plt.xlabel("Height", fontsize=AXIS_LABEL_FONT_SIZE)
                    plt.ylabel("EI", fontsize=AXIS_LABEL_FONT_SIZE)
                    plt.ylim(1e-6, 5)
                    plt.yscale('log')
                    plt.xticks(fontsize=TICK_FONT_SIZE)
                    plt.yticks(fontsize=TICK_FONT_SIZE)
                    plt.legend()

                    plt.tight_layout()
                    file_name = os.path.join(output_folder, f"iteration_{i}.png")
                    plt.savefig(file_name, dpi=300, bbox_inches="tight")
                    plt.close()
                    print(f"Plot for iteration {i} saved as {file_name}")

                if self.dimension == 2 and i % 10 == 0:
                    mu_norm, cov_norm = self.compute_posterior(x_candidates)
                    std_norm = torch.sqrt(torch.diag(cov_norm))

                    y_range = (self.global_y_max - self.global_y_min) + 1e-12
                    mu_rescaled = mu_norm * y_range + self.global_y_min
                    std_rescaled = std_norm * y_range

                    ei_vals = self.expected_improvement(x_candidates).cpu().numpy()

                    X1, X2 = np.meshgrid(x1_candidates.cpu().numpy(), x2_candidates.cpu().numpy())
                    mu_rescaled_grid = mu_rescaled.cpu().numpy().reshape(X1.shape)
                    ei_vals_grid = ei_vals.reshape(X1.shape)
                    ei_vals_grid = np.maximum(ei_vals_grid, 1e-14)

                    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

                    # LEFT: GP Posterior (Contour Plot)
                    ax = axes[0]
                    ax.set_title(f"Iteration {i} - GP Posterior", fontsize=TITLE_FONT_SIZE)
                    contour = ax.contourf(X1, X2, mu_rescaled_grid, cmap='viridis')
                    evaluated_points = self.X.cpu().numpy()
                    ax.scatter(
                        evaluated_points[:, 1], evaluated_points[:, 0],
                        color="blue", marker='o', label="Last 10 Points" if len(evaluated_points) > 10 else "Evaluated Points"
                    )
                    if len(evaluated_points) > 10:
                        ax.scatter(
                            evaluated_points[-10:, 1], evaluated_points[-10:, 0],
                            color="yellow", marker='o', label="Last 10 Points"
                        )
                    ax.scatter(x_new[0, 1].item(), x_new[0, 0].item(), color="red", marker='X', label="Next Point", s=100)
                    ax.set_xlabel('A', fontsize=AXIS_LABEL_FONT_SIZE)
                    ax.set_ylabel('y', fontsize=AXIS_LABEL_FONT_SIZE)
                    ax.tick_params(axis='both', which='major', labelsize=TICK_FONT_SIZE)
                    colorbar = fig.colorbar(contour, ax=ax, label='Objective Function Value')
                    colorbar.ax.tick_params(labelsize=TICK_FONT_SIZE)

                    ax.legend()

                    # RIGHT: EI (Contour Plot)
                    ax = axes[1]
                    ax.set_title(f"Iteration {i} - EI | Max: {best_ei:.4f}", fontsize=TITLE_FONT_SIZE)
                    contour = ax.contourf(
                        X1, X2, ei_vals_grid, cmap='magma', norm=LogNorm(vmin=1e-14, vmax=0.1)
                    )
                    ax.scatter(
                        evaluated_points[:, 1], evaluated_points[:, 0],
                        color="blue", marker='o',alpha = 0.5, label="Last 10 Points" if len(evaluated_points) > 10 else "Evaluated Points"
                    )
                    if len(evaluated_points) > 10:
                        ax.scatter(
                            evaluated_points[-10:, 1], evaluated_points[-10:, 0],
                            color="yellow", marker='o', label="Last 10 Points"
                        )

                    ax.scatter(x_new[0, 1].item(), x_new[0, 0].item(), color="red", marker='X', label="Next Point", s=100)
                    ax.set_xlabel('A', fontsize=AXIS_LABEL_FONT_SIZE)
                    ax.set_ylabel('y', fontsize=AXIS_LABEL_FONT_SIZE)
                    ax.tick_params(axis='both', which='major', labelsize=TICK_FONT_SIZE)
                    colorbar = fig.colorbar(contour, ax=ax, label='Expected Improvement', format=LogFormatter())
                    colorbar.ax.tick_params(labelsize=TICK_FONT_SIZE)
                    ax.legend()

                    # Save plot
                    file_name = os.path.join(output_folder_2, f"iteration_{i}.png")
                    plt.tight_layout()
                    plt.savefig(file_name, dpi=300, bbox_inches="tight")
                    plt.close()
                    print(f"Plot for iteration {i} saved as {file_name}")

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Total optimization time: {elapsed_time:.2f} seconds")




        return self.log_df

### Gaussian Process Kernels

In Gaussian Processes (GPs), the choice of kernel function $k(x, x')$ is crucial as it defines the covariance structure of the function being modeled. The kernels used in this study include:

#### 1. Squared Exponential (RBF) Kernel
The squared exponential kernel, also known as the Radial Basis Function (RBF) kernel, is defined as:

$$
k(x, x') = \sigma_f^2 \exp\left( -\frac{||x - x'||^2}{2 l^2} \right)
$$

where:
- $\sigma_f^2$ is the signal variance.
- $l$ is the length-scale, which determines the smoothness of the function.
- $||x - x'||^2$ is the squared Euclidean distance between input points.

This kernel ensures smooth and infinitely differentiable functions.

#### 2. Matérn $\nu = 0.5$ Kernel
The Matérn kernel with $\nu = 0.5$ (also known as the Ornstein-Uhlenbeck process) is defined as:

$$
k(x, x') = \sigma_f^2 \exp\left(-\frac{||x - x'||}{l}\right)
$$

This kernel models less smooth functions than the squared exponential kernel, allowing for sharp changes.

#### 3. Matérn $\nu = 1.5$ Kernel
For $\nu = 1.5$, the Matérn kernel takes the form:

$$
k(x, x') = \sigma_f^2 \left(1 + \frac{\sqrt{3} ||x - x'||}{l} \right) \exp\left(-\frac{\sqrt{3} ||x - x'||}{l} \right)
$$

This kernel results in functions that are once differentiable.

#### 4. RBF Kernel with Basis Functions
The Radial Basis Function (RBF) kernel with basis functions is given by:

$$
k(x, x') = \frac{1}{\alpha} \phi(x)^\top \phi(x')
$$

where $\phi(x)$ is a set of Gaussian basis functions:

$$
\phi(x) = \exp\left(-\frac{(x - \mu_m)^2}{2 s^2}\right)
$$

where:
- $\mu_m$ are the basis function centers.
- $s$ is the scale parameter controlling spread.
- $\alpha$ is a scaling factor.


Each kernel has distinct properties and the choice depends on the function smoothness and complexity requirements.

In [8]:
def squared_exponential(X1, X2, hyperparams):
    """
    Squared Exponential (RBF) Kernel.
    
    This kernel function is also known as the Radial Basis Function (RBF) kernel.
    It is widely used in Gaussian Process regression due to its smoothness 
    properties and ability to model infinitely differentiable functions.

    Args:
        X1 (torch.Tensor): First set of points, shape [n1, d].
        X2 (torch.Tensor): Second set of points, shape [n2, d].
        hyperparams (dict): Dictionary containing:
            - 'sig_f' (float): The signal variance (scale factor).
            - 'length' (float): The characteristic length-scale of the kernel.

    Returns:
        torch.Tensor: Kernel matrix of shape [n1, n2].
    """
    sig_f = hyperparams["sig_f"]
    length = hyperparams["length"]

    sqdist = torch.cdist(X1, X2) ** 2
    return sig_f**2 * torch.exp(-sqdist / (2 * length**2))

def matern_0_5(X1, X2, hyperparams):
    """
    Matérn kernel for ν = 0.5 (simplified form).

    Args:
        X1 (torch.Tensor): First set of points, shape [n1, d].
        X2 (torch.Tensor): Second set of points, shape [n2, d].
        hyperparams (dict): Dictionary containing:
            - 'sig_f' (float): The signal variance (scale factor).
            - 'length' (float): The characteristic length-scale of the kernel.

    Returns:
        torch.Tensor: Kernel matrix of shape [n1, n2].
    """
    sig_f = hyperparams["sig_f"]
    length = hyperparams["length"]

    dist = torch.cdist(X1, X2)

    return sig_f**2 * torch.exp(-dist / length)

def matern_kernel_nu_1_5(X1, X2, hyperparams):
    """
    Matérn kernel for ν = 1.5.

    Args:
        X1 (torch.Tensor): First set of points, shape [n1, d].
        X2 (torch.Tensor): Second set of points, shape [n2, d].
        hyperparams (dict): Dictionary containing:
            - 'sig_f' (float): The signal variance (scale factor).
            - 'length' (float): The characteristic length-scale of the kernel.

    Returns:
        torch.Tensor: Kernel matrix of shape [n1, n2].
    """
    sig_f = hyperparams["sig_f"]
    length = hyperparams["length"]

    dist = torch.cdist(X1, X2)
    
    sqrt_3 = torch.sqrt(torch.tensor(3.0))
    factor = sqrt_3 * dist / length
    matern = (1 + factor) * torch.exp(-factor)

    return sig_f**2 * matern

def RBF_kernel(X1, X2, hyperparams):
    """
    Computes the kernel function k(x, x') based on RBF features.
    
    Args:
        X1 (torch.Tensor): Input tensor of shape (n_samples_1, n_features).
        X2 (torch.Tensor): Input tensor of shape (n_samples_2, n_features).
        hyperparams (dict): Dictionary containing:
            - 'alpha' (float): Scaling factor for the kernel.
            - 'M' (int): Number of basis functions.
            - 'length' (float): Scale parameter for the Gaussian basis functions.
    
    Returns:
        torch.Tensor: Kernel matrix of shape (n_samples_1, n_samples_2).
    """
    alpha = hyperparams['alpha']
    M = hyperparams['M']
    s = hyperparams['length']
    
    mu_m = torch.linspace(1, 5, M)  # Adapted to input space (1,5)

    def compute_phi(X, mu_m, s):
        phi = torch.exp(-((X.unsqueeze(-1) - mu_m) ** 2) / (2 * (s ** 2)))
        return phi  

    phi_X1 = compute_phi(X1, mu_m, s)  
    phi_X2 = compute_phi(X2, mu_m, s)  

    phi_X1 = phi_X1.view(X1.size(0), -1) 
    phi_X2 = phi_X2.view(X2.size(0), -1)  

    kernel_matrix = (1 / alpha) * (phi_X1 @ phi_X2.T)  

    return kernel_matrix

## Initial Hyperparameter Tuning

There are three main methods for hyperparameter optimization: random search, grid search, and Bayesian optimization. Among these, Bayesian optimization was chosen for its scalability and efficiency. It was implemented using the skopt.gp_minimize function, which approximates the objective function with a Gaussian process and leverages Bayesian optimization to find the best values. This function is ideal for searching the multidimentional space of hyperparamenters efficiently, and it allows to define both a continuous and discrete searchspace, as well as the kernel search dictionary.

Ultimately, it was decided to reduce the problem's dimensionality and optimize only the kernel-related hyperparameters. The optimization was limited to four kernels: Matern 0.5, Matern 1.5, Squared Exponential, and Radial Basis Function. These kernels were compatible with the Cholesky decomposition used in the Bayesian optimization code and were considered well-suited for the problem.



### Optimisation function

 Designing the optimization function for hyperparameter tuning was a challenge in itself. Four different penalty options were explored:

1 Penalty for high Lagrange value

2 Penalty for high Lagrange value and number of iterations

3 Penalty for high Lagrange value and time taken

4 Penalty for high Lagrange value, number of iterations, and time taken

Here, the number of iterations refers to how long the Bayesian optimizer took to converge, while the Lagrange value represents the best performance achieved, accounting for bridge weight and frequency constraints.

Time penalty was removed since it correlates with the number of iterations, allowing the optimization to focus more directly on performance and minimizing weight. A further comparison was conducted to evaluate the impact of penalizing the number of iterations. This was tested using the Squared Exponential kernel.

With the iteration penalty applied, optimal hyperparameters tended to concentrate at the boundaries of the search space, rarely taking intermediate values. Without this penalty, the hyperparameters were more evenly distributed across the range. Based on these findings, it was decided to proceed with only the Lagrange penalty, leading to the final optimization function.


In [ ]:
def hyperparam_optim_wrapper(hyperparams_list, verbose=False):
    hyperparams = hyperparam_list_to_dict(hyperparams_list)
    dimension = 2
    optimizer_bridge = BayesianOptimizer(dimension, bounds, hyperparams, objective_function)
    optimizer_bridge.initialize(n_initial_points=2)
    log_df = optimizer_bridge.optimize(n_iterations=100, plot=False)
    best_y_index = log_df['Best_Y'].idxmin()
    y = log_df.loc[best_y_index, 'Best_Y']
    if verbose:
        print(f'Optimal height: y={y:.2f} m')
    lagrange=y
    return lagrange/100

Another key aspect of this function is that it sets a maximum limit of 100 iterations for the Bayesian optimizer. If the stopping criterion is not triggered earlier, the optimizer will automatically stop at the 100th iteration.

#### Searchspace

The searchspace for hyperparameters was defined as follows:

In [ ]:
search_space = [
    skopt.space.Real(0.1, 5, name='length', prior='log-uniform'),  # Log-uni distribution for lr
    skopt.space.Real(0.01, 10, name='alpha', prior='log-uniform'),
    skopt.space.Integer(1, 10, name='M'),
    skopt.space.Categorical(list(kernels.kernels_dict.values())),
    skopt.space.Real(0.01, 1, name='sig_f')
]

### Running Hyperparameters Optimiser

Now that the objective function and bounds for the hyperparameter search have been defined, we can proceed with formulating the hyperparameter optimizer. The optimizer was configured to perform 100 function evaluations to improve the chances of finding a global minimum. Since only the length and sig_f hyperparameters are actively used in the optimization, the search space is effectively two-dimensional. Based on previous experience with Bayesian optimization for the bridge problem, 100 evaluations were deemed sufficient.

To introduce randomness into the search process and increase the likelihood of escaping local minima, the random_state parameter was left unset (random_state=None). This ensures that each run of the optimizer explores different regions of the search space, making the results less dependent on specific initial conditions.


In [ ]:
result = skopt.gp_minimize(
                func=hyperparam_optim_wrapper,   # Objective function
                dimensions=search_space,        # Search space
                n_calls=100,                      # Number of evaluations
                n_random_starts=5,              # Number of random initial points
                random_state=None,            # Different seed for each run
                verbose=False                   # Set to True if you want detailed logs
            )

This analysis was conducted for four different kernels, with each kernel being tested 10 times for both 1D and 2D cases. The results are summarized below. To interpret the distribution and performance of the hyperparameters across multiple runs, a boxplot approach was chosen. Boxplots are particularly effective for understanding the convergence behavior, as they highlight the interquartile range (IQR), which gives insight into the spread and consistency of the results. The IQR also provides a clear indication of how much variability exists within the results, making it easier to assess whether the optimization process is stabilized.

Furthermore, the boxplot clearly displays the median value of the results, which serves as an important reference point. The median can be used as a starting hyperparameter value, giving us a solid baseline for further tuning and refinement. This approach not only enables us to observe the overall trend but also allows us to identify potential outliers and extreme values.

| **Kernel**               | **Parameter**   | **Minimum**   | **Q1**    | **Median**    | **Q3**        | **Maximum**   |
|--------------------------|-----------------|---------------|-----------|---------------|---------------|---------------|
| **1D Matern 0.5**        | length scale    | 0.1669679031  | 0.423563471 | 1.243886678   | 3.934174402   | 4.433075298   |
|                          | kernel scale    | 0.06828955908 | 0.2553031999 | 0.2783115734  | 0.4289226336  | 0.6788941316  |
| **1D Matern 1.5**        | length scale    | 0.2310861223  | 0.8165734606 | 1.609465864   | 4.330440765   | 5             |
|                          | kernel scale    | 0.0831302186  | 0.2132119631 | 0.3656775658  | 0.7275127041  | 0.8921444765  |
| **1D Squared exponential** | length scale    | 0.1           | 0.1       | 0.2508005461  | 0.5653789967  | 3.185474501   |
|                          | kernel scale    | 0.01          | 0.092427494 | 0.1793526587  | 0.355301434   | 0.8480936205  |
| **1D Radial Basis Function** | length scale    | 0.1042479791  | 0.2941014119 | 0.3220311005  | 1.183575599   | 1.944067924   |
|                          | alpha           | 0.01          | 0.05523178748       | 0.4960094711  | 2.31673295    | 9.191279891   |
|                          | M               | 6             | 7         | 8             | 9             | 10            |
| **2DMatern 0.5**         | length scale    | 0.1           | 0.1077170347 | 0.2834078635  | 2.141074521   | 2.888188723   |
|                          | kernel scale    | 0.07088517591 | 0.4975153643 | 0.9405388924  | 1             | 1             |
| **2D Matern 1.5**        | length scale    | 0.1501793743  | 0.8991476863 | 2.42728109    | 3.849900309   | 5             |
|                          | kernel scale    | 0.01          | 0.1885102096 | 0.3102989121  | 0.552743954   | 0.9635908927  |
| **2D Squared exponential** | length scale    | 0.4432878479  | 1.088323546 | 1.195464378   | 1.244799528   | 1.309108542   |
|                          | kernel scale    | 0.01085110859 | 0.5458963669 | 0.7363841932  | 0.7680583266  | 1             |
| **2D Radial Basis Function** | length scale    | 0.1726782106  | 0.3934751097 | 0.5919010207  | 0.8707366376  | 1.273689816   |
|                          | alpha           | 0.01          | 0.01      | 0.01181149163 | 4.353558766   | 5             |
|                          | M               | 8             | 8         | 8             | 9.75          | 10            |


The following conclusions can be made about the differnt kernels based on the data above:
##### 1D Case:
- **Matern 0.5**:
  - Length scale: Range 0.16 - 4.43 (a very wide range, no concentration at boundaries).
  - Kernel scale: Range 0.068 - 0.68 (fairly wide range, no concentration at boundaries).

- **Matern 1.5**:
  - Length scale: Range 0.23 - 5 (a wide range, rarely falls into boundary treshold).
  - Kernel scale: Range 0.08 - 0.89 (very wide range)


- **Squared Exponential**:
  - Length scale: Range 0.1 - 3.19 (often falls into the lower boundary, concentrated in that area).
  - Kernel scale: Range 0.01 - 0.85 ( rarely falls into boundary treshold, half of the data is below 0.2)

- **Radial Basis Function (RBF)**:
  - Length scale: Range 0.1 - 1.94 (No concentration at boundaries, relatively small range.)
  - Alpha: Range 0.01 - 9.19 (alpha can significantly impact performance, and has a large range, but 50% of the data is concentrated below 1).
  - M: IQ Range 7-9, (very narrow).

##### 2D Case:
- **Matern 0.5**:
  - Length scale: Range: 0.1 -  2.89
  - Kernel scale: 0.07 to 1.

- **Matern 1.5**:
  - Length scale: 0.15 to 5.
  - Kernel scale: 0.01 to 0.96.


- **Squared Exponential**:
  - Length scale: Range 0.44 - 1.31. IQ Range 1.09 - 1.24 (very narrow, no fallls into bounds, stable)
  - Kernel scale: Range 0.01 - 1. IQ Range 0.55 - 0.77 (reasonable narrow range, falls into bounds sometimes)


- **Radial Basis Function (RBF)**:
  - Length scale: Reange 0.39 - 0.87, IQ range 0.54 - 0.76 (very narrow range results are concentrated and converge  around median)
  - Alpha: Range 0.01 - 5 (Data either falls on the higher or inner bound of the search space).
  - M: IQ range 8 - 9.75 (Data is concentrated around 8 with some outliers on higher end).



### Results


Below the results are given, described and discussed for the various dimensionality problems. 

#### 1D

**Table of hyperparameters:**

| Parameter              | Value       |
|------------------------|-------------|
| $\sigma_f$ (sig_f)     | 0.2         |
| Length scale           | 1.0         |
| Noise                  | $1 \times 10^{-10}$ |
| $\beta$                | 5.0         |
| Number of suggestions  | 10          |
| Constraint weight      | 100         |
| Adam iterations        | 20          |
| Initial points         | 2           |
| Seed                  | 42          |

**Results:**  
- Optimal height: **1.62 m**  
- Weight: **344.52 kg**  
- Eigenvalues: **[12.38, 33.43, 51.72]** 

The total optimization time was 7.83 seconds an the process was run on a CPU. It can clearly be seen that the boundary conditions were not fulfilled, as this was impossible by only changing the height of the nodes. The height is therefore the optimal value for the lowest weight while being as close as possible to the boundary conditions.

**Discussion:**

The hyperparameters were chosen to balance exploration and exploitation in the Bayesian optimization process. The initial points are set to 2 to ensure the optimization starts with sufficient prior data for the Gaussian Process (GP) to build its initial posterior distribution.

The following figures (3, 4) summarize the results of the 1D optimization process:

1. Best Y over Iterations (Left):
   This plot shows the progression of the best output value of the objective function ($Y$) over iterations. The sharp initial decrease indicates that the optimizer rapidly identifies a promising region of the search space, after which improvements become more incremental.

2. Iteration Y over Iterations (Middle):
   This figure tracks the output of the objective function for the proposed candidate at each iteration. The oscillations correspond to the exploration phase, where the optimizer tries different areas of the search space. It can be seen that when the hyperparameters optimise the output of the objective function is extremely high and the expected improvement is extremely low. These most likely occur due to temporary instability or shifts in the GP model as it adjusts to new hyperparameter values. Overfitting or increased model confidence in less-explored regions can lead to exaggerated predictions.

3. Expected Improvement (EI) over Iterations (Right):
   The expected improvement function decreases as the optimizer approaches convergence. Spikes enlarging the EI occur when the optimizer finds regions with high uncertainty or potential for improvement.

   ![1D EI Plot](Images/1D_EI_plot.png)
   Figure 3: Logarithmic plot of the best out value ($Y$) over iterations (left), and logarithmic plot of EI over iterations (right).

4. The 1D optimization process is further visualized in the accompanying GIF, Figure 4 below. The left panel dynamically updates the GP posterior as new points are evaluated. it displays the GP mean, confidence interval and the next suggested point based on the EI which is shown on the right. The highest, or closed to the highest based on the Adam optimizer, determines the next candidate point. It can be concluded that the BO works as intended and finds the minimum.

   ![1D_GIF](Images/1D_gif.gif) 

   Figure 4: GIF of the dynamically updated GP (left), and a GIF of the corresponding EI and the next suggested point (right).  

5. Hyperparameter Evolution:
   Figure 5 illustrates the evolution of key hyperparameters during optimization. 
   - $\sigma_f$: Represents the signal variance of the GP and shows a gradual increase, reflecting the model's increasing confidence in predicting function outputs.
   - Length scale: Adjusts the smoothness of the GP kernel, adapting as the optimizer explores the search space.
   - $\beta$: The exploration factor decreases over time, promoting convergence by reducing the emphasis on exploring uncertain regions.
   
   ![1D Hyperparameter Plot](Images/1D_hyperparameter_plot.png)
   Figure 5: Plots of the evoluation of hyperparameters $\sigma_f$ (left) and $\beta$ (right) over iterations.

**Summary:**
The 1D optimization demonstrates effective convergence, balancing exploration and exploitation through the EI acquisition function. The adaptive nature of the hyperparameters ensures the GP evolves to fit the observed data, allowing the optimizer to refine its predictions and converge on the optimal solution efficiently.




#### 2D
**Table of hyperparameters:**

| Parameter              | Value       |
|------------------------|-------------|
| $\sigma_f$ (sig_f)     | 0.2         |
| Length scale           | 1.0         |
| Noise                  | $1 \times 10^{-10}$ |
| $\beta$                | 5.0         |
| Number of suggestions  | 10          |
| Constraint weight      | 500         |
| Adam iterations        | 20          |
| Initial points         | 100         |
| Seed                  | 42           |

**Optimal Results:**  
- Optimal height: **1.99 m**  
- Optimal cross-section: **2.60 cm²**  
- Weight: **409.56 kg**  
- Eigenvalues: **[20.03, 49.93, 75.36]**  

The total optimization time was **161.01 seconds** and the process was run on a **CPU**. By adding the crossection to the input values the solution satisfies to the boundary conditions. It must be noted that was with a constraint weight of 500. If we would keep the constraint weight the same as in the 1D optimisation, as shown in the presentation, the first eigenvalue would be slightly below the boundary of 20, as can be seen below. This shows the effect that the constraint weight can have on the optimizer. In both cases the BO finds the global minimum, based on running a grid calculation on the objective function, but the difference in output shows the importance of the correct constraint weight. The discussion is based on the optimization with a constraint weight of 100. 

**Suboptimal Results:**  
- Optimal height: **1.91 m**  
- Optimal cross-section: **2.64 cm²**  
- Weight: **408.25 kg**  
- Eigenvalues: **[19.92, 50.58, 76.67]**  

**Discussion:**

The hyperparameters for the 2D optimization were selected to strike a balance between exploration and exploitation. A higher number of initial points (100) was used to ensure the GP had sufficient data to construct an accurate posterior distribution in the higher-dimensional search space.

The following figures (6, 7) summarize the results of the 2D optimization process:

1. *Best Y over Iterations (Left):*  
   This plot illustrates the progression of the best observed output value ($Y$) over iterations. A rapid initial improvement is observed as the optimizer identifies promising regions, followed by slower, incremental improvements.

2. *Iteration Y over Iterations (Middle):*  
   This figure tracks the output of the objective function for each iteration's proposed candidate. The oscillations highlight the balance between exploration of new areas and exploitation of known promising regions.

3. *Expected Improvement (EI) over Iterations (Right):*  
   The expected improvement decreases as the optimization converges. Spikes in EI reflect moments when the optimizer identifies regions of high uncertainty or potential for improvement.

   ![2D EI Plot](Images/2D_EI_plot.png)
   Figure 6: Logarithmic plot of the best out value ($Y$) over iterations (left), and logarithmic plot of EI over iterations (right).

4. *Optimization Process Visualization (GIF):*  
   The accompanying GIF provides a dynamic visualization of the 2D optimization process.  
   - The left panel shows the GP posterior with the GP mean and confidence intervals evolving as new points are evaluated. Evaluated points are represented by dots, with the next point suggested by the optimizer marked.  
   - The right panel displays the Expected Improvement (EI), showing how the optimizer selects the next point with the highest improvement potential.  

   ![2D Optimization GIF](Images/2D_gif_option2.gif)

   Figure 7: GIF of the dynamically updated GP (left), and a GIF of the corresponding EI and the next suggested point (right).
   

5. *Hyperparameter Evolution:*  
   These plots illustrate the evolution of key hyperparameters during optimization.  
   - $\sigma_f$: Represents the signal variance of the GP and shows a gradual increase, reflecting the model's increasing confidence in predicting function outputs.  
   - Length scale: Adjusts the smoothness of the GP kernel, adapting as the optimizer explores the search space.  
   - $\beta$: The exploration factor decreases over time, promoting convergence by reducing the emphasis on exploring uncertain regions.  

   ![2D Hyperparameter Plot](Images/2D_hyperparameter_plot.png)

   Figure 8: Hyperparameter tuning

**Summary:**  
The 2D optimization demonstrates effective convergence, balancing exploration and exploitation through the EI acquisition function. The GP dynamically evolves to fit the observed data, allowing the optimizer to efficiently refine its predictions and identify the optimal solution.

#### 6D
During the initial optimization process for the 6D case, the run was conducted on the CPU using a TU Delft server. This was an oversight, as the use of a GPU would have significantly reduced the computation time. After realizing this mistake during the presentation, we decided to rerun the optimization using the Kaggle GPU environment to leverage its faster computational capabilities.

Additionally, for the GPU run, we chose to extend the number of iterations and increase the Adam iterations from 20 to 100. This adjustment was made to allow for more thorough exploration and better convergence of the optimization process. The increased computational efficiency of the GPU environment enabled us to perform these additional iterations in a fraction of the time required on the CPU. Below is a comparison of the hyperparameter settings and execution details for both runs.


**Comparison of Hyperparameter Settings for 6D Optimization:**

| Parameter                  | CPU/Presentation Run                             | GPU/Kaggle Run                              |
|----------------------------|------------------------------------------------|--------------------------------------------|
| $\sigma_f$ (sig_f)         | 0.2                                            | 0.2                                        |
| Length scale               | 1.0                                            | 1.0                                        |
| Noise                      | $1 \times 10^{-8}$                             | $1 \times 10^{-8}$                         |
| $\beta$                    | 5.0                                            | 5.0                                        |
| Number of suggestions      | 10                                             | 10                                         |
| Constraint weight          | 100                                            | 100                                        |
| Adam iterations            | 20                                             | **100**                                    |
| Beta lowering iteration    | After **150**                                  | After **500**                              |
| Hyperparameter updates     | Every 10th iteration                           | Every 10th iteration, then after 150 every 50th **until 500** |
| Depth                      | **$1 \times 10^{-20}$**                        | **$1 \times 10^{-30}$**                    |
| Running time               | **4 hours**                                    | **~1 hour**                                |
| Device                     | **CPU**                                        | **GPU**                                    |
| Max allowed iteraitions    | **600**                                        | **1000**                                   |
| Initial values X           |1000                                            | 1000                                       |

**Highlighted differences:**  
- The Adam iterations for the GPU run were increased to 100 for better convergence.  
- Beta lowering occurred much later (iteration 500) in the GPU run compared to the CPU run. This was to provide more iterations for explaration before shifting to exploitation. 
- Hyperparameter update frequency for the GPU run followed a similar pattern to the CPU but extended to iteration 500. After which is stopped to motivate convergence.  
- The running time for the GPU run was significantly faster, completing in about 1 hour compared to 4 hours on the CPU, even though 100 Adam iterations were used.
- The maximum allowed iterations was increased from 600 to 1000. 

**Comparison of CPU and GPU Results for 6D Optimization**

The 6D optimization process was conducted twice, first on a CPU (presentation run) and later on a GPU in the Kaggle environment. The different computational resources, number of iterations, and hyperparameter schedules produced distinct outcomes.

**Optimization Results:**

| Metric                     | CPU Run                                      | GPU Run                                    |
|----------------------------|----------------------------------------------|--------------------------------------------|
| **Optimal Heights ($y_i$)**    | [1.376, 2.009, 2.068, 2.511, 2.205]       | [1.0, 1.381, 1.926, 2.192, 2.283]          |
| **Optimal Crossection ($A$)**  | 2.546 cm²                                 | 2.256 cm²                                  |
| **Weight (kg)**                | 407.48                                    | **385.91**                                 |
| **Eigenvalues**                | [21.11, 45.59, 70.50]                     | [21.36, 41.61, 71.98]                      |
| **Number of Iterations**       | ~360                                      | **1000**                                   |
| **Beta Lowering**              | After 150 iterations                      | After 500 iterations                       |
| **Stopped Tuning**             | After 150 iterations                      | Tuned every 50 iterations post-500         |

**Discussion of Results:**

- Significantly Lower Weight on the GPU Run:  
  The GPU optimization discovered a design with a weight of 385.91 kg, which is the lowest among all tested runs in all dimensions. This could be attributed to both the increased number of iterations** (1000 vs. ~360) and the higher Adam iterations (allowing more thorough local optimization of candidate points).

- Longer Exploration  
  In the GPU run, beta lowering started much later (iteration 500), providing more time for the optimizer to explore. This likely contributed to finding a lighter design while still respecting the structural constraints, as evidenced by the final eigenvalues.

- Hyperparameter Adjustments  
  The GPU run continued hyperparameter tuning at regular intervals, including beyond iteration 150, refining the Gaussian Process model’s fit to the data throughout more of the optimization process.



1. **CPU Run (6D) Plots**  
  
    Figure 9 below sumarizes the results of the 6D optimization process ran on CPU. 

  - *Best Y over Iterations (Left):*  
      This plot illustrates the progression of the best observed output value ($Y$) over iterations. A rapid initial improvement is observed as the optimizer identifies promising regions, followed by slower, incremental improvements.

  - *Iteration Y over Iterations (Middle):*  
     This figure tracks the output of the objective function for each iteration's proposed candidate. The oscillations highlight the balance between exploration of new areas and exploitation of known promising regions.

  - *Expected Improvement (EI) over Iterations (Right):*  
     The expected improvement decreases as the optimization converges. Spikes in EI reflect moments when the optimizer identifies regions of high uncertainty or potential for improvement.

      ![CPU 6D Plots](Images/6D_EI_plot_poster.png)
   
      Figure 9: Logarithmic plots of the best output value ($Y$) over iterations (left), the output value ($Y$) per iteration (middle), and the EI over iterations (right) ran on CPU.

2. **GPU Run (6D) Plots**  

    Figure 10 below sumarizes the results of the 6D optimization process ran on GPU.

   ![GPU 6D Plots](Images/6D_EI_plot_GPU.png)
   
    Figure 10: Logarithmic plots of the best output value ($Y$) over iterations (left), the output value ($Y$) per iteration (middle), and the EI over iterations (right) ran on GPU.

3. **CPU vs. GPU Hyperparameters**  

  Figures 11 and 12 below display the evolution of the hyperparameters during the optimization process ran on CPU and GPU.   
- *$\sigma_f$ over Iterations (Left):*  
  This plot displays the evolution of the kernel scale ($\sigma_f$) over iterations. Eventually, the kernel scale converges to ~0.28.  

- *Length Scale over Iterations (Middle):*  
  This plot displays the evolution of the length scale of the kernel over iterations. The length scale converges to ~1.9.  

- *$\beta$ over Iterations (Right):*  
  This plot shows the evolution of $\beta$ over iterations. After iteration 150, $\beta$ is reduced with a constant reduction factor of 0.75 per 25 iterations.  

   **CPU**
   ![CPU 6D Hyperparams](Images/6D_hyperparameter_plot_poster.png)

   Figure 11: Plots showing the evolution of the kernel scale over iterations (left), length scale over iterations (middle), and beta over iterations (right) ran on CPU.

   **GPU** 
   ![GPU 6D Hyperparams](Images/6D_hyperparameter_plot_GPU.png)

   Figure 12: Plots showing the evolution of the kernel scale over iterations (left), length scale over iterations (middle), and beta over iterations (right) ran on GPU.

**Conclusion:**
Running the optimization on a GPU allowed for more iterations and more extensive hyperparameter tuning. This deeper exploration of the design space produced a significantly lower weight (385.91 kg) compared to the CPU run (407.48 kg), while still maintaining comparable eigenvalues. The higher Adam iterations also contributed to refining candidate points more thoroughly, indicating the value of greater computational resources and a more extended optimization horizon for complex, high-dimensional problems.

#### 19D

The 19D optimization follows the same hyperparameter settings as the 6D runs, with the key difference being that the number of initial points was doubled to 2000 to better handle the higher dimensionality. By increasing the initial sample size, the optimizer gains more information at the start, which can be crucial for effective exploration in a 19-dimensional design space. The same beta lowering schedule and hyperparameter update strategy were applied, but the complexity of the problem required extended computation and thorough exploration to achieve meaningful results.


 Metric               | CPU Run (Stopped at 449 Iterations)                | GPU Run (1000 Iterations)                      |
|----------------------|----------------------------------------------------|----------------------------------------------------------------|
| *Weight (kg)*      | 406.97                                              | 404.15                                                        |
| *Eigenvalues*      | [20.51, 40.20, 63.95]                        | [24.67, 40.71, 64.84]                                    |
| *Initial Points*   | 2000                                               | 2000                                                           |


Below are the 5 height parameters $(y_1 \ldots y_5)$ and 14 cross-sectional area parameters $(A_1 \ldots A_{14})$ for both the CPU and GPU runs. ”

| Index | Parameter | CPU Value  | GPU Value         |
|-------|-----------|------------|-------------------|
| 1     | *y1*    | 1.38       | 1.00              |
| 2     | *y2*    | 2.09       | 1.59              |
| 3     | *y3*    | 2.56       | 1.76              |
| 4     | *y4*    | 2.52       | 1.97              |
| 5     | *y5*    | 2.95       | 2.02              |
| 6     | *A1*    | 1.85       | 2.38              |
| 7     | *A2*    | 2.31       | 4.92              |
| 8     | *A3*    | 2.59       | 5.00              |
| 9     | *A4*    | 2.90       | 4.75              |
| 10    | *A5*    | 2.19       | 1.00              |
| 11    | *A6*    | 3.69       | 2.09              |
| 12    | *A7*    | 1.73       | 5.00              |
| 13    | *A8*    | 3.45       | 2.52              |
| 14    | *A9*    | 2.09       | 2.97              |
| 15    | *A10*   | 2.11       | 5.00              |
| 16    | *A11*   | 2.67       | 1.00              |
| 17    | *A12*   | 1.50       | 1.86              |
| 18    | *A13*   | 2.91       | 2.38              |
| 19    | *A14*   | 1.23       | 3.54              |


**Discussion of Results:**

- Differences Between CPU and GPU Runs:  
    The CPU run stopped after 449 iterations, whereas the GPU run kept running untill the maximum number of iterations was reached at 1000. The GPU result found a slightly lower mass (404.14 kg vs. 406.97 kg).  
  - Beta and Hyperparameter Tuning: The CPU run may have reduced beta sooner or stopped hyperparameter updates, leading to a more exploitative phase. This can explain the occasional spikes in the best $Y$ (the CPU’s best value occasionally goes up).  
  - Possibility for Better Results: Allowing the CPU run to continue hyperparameter tuning or run for additional iterations might produce a closer or improved result.

- Comparison with 6D:  
  The 6D optimizer achieved a lower mass than either 19D result. Bayesian optimization is known to struggle more as dimensionality increases, requiring more data points and iterations to converge effectively. With 19 dimensions, exploration becomes more challenging and the GP surrogate may be less reliable without substantially more iterations.


1. **CPU and GPU Run (19D) Plots**

    Figure 13 below sumarizes the results of the 19D optimization process ran on CPU. 

  - *Best Y over Iterations (Left):*  
      This plot illustrates the progression of the best observed output value ($Y$) over iterations. A rapid initial improvement is observed as the optimizer identifies promising regions, followed by slower, incremental improvements.

  - *Iteration Y over Iterations (Middle):*  
     This figure tracks the output of the objective function for each iteration's proposed candidate. The oscillations highlight the balance between exploration of new areas and exploitation of known promising regions. After iteration 300, the output of the objective function increases. This is likely due to the hyperparameters not being fully optimized yet since they are halted at iteration 150, the solution space is not very well captured yet. Another possibility could be that the beta is lowered too quickly and reaches ~0, the uncertainty would then be equal for the whole solution space. The acquistion would then suggest random points. 

  - *Expected Improvement (EI) over Iterations (Right):*  
     The expected improvement decreases as the optimization converges. Spikes in EI reflect moments when the optimizer identifies regions of high uncertainty or potential for improvement.  

   ![CPU 19D Plots](Images/19D_EI_plot_poster.png)

   Figure 13: Logarithmic plots of the best output value ($Y$) over iterations (left), the output value ($Y$) per iteration (middle), and the EI over iterations (right) ran on CPU.
   
   Figure 14 below sumarizes the results of the 19D optimization process ran on GPU.

   ![GPU 19D Plots](Images/19D_EI_plot_GPU.png)
   
   Figure 14: Logarithmic plots of the best output value ($Y$) over iterations (left), the output value ($Y$) per iteration (middle), and the EI over iterations (right) ran on GPU.

1. **CPU vs. GPU Hyperparameters**  

    Figures 15 and 16 below display the evolution of the hyperparameters during the optimization process ran on CPU and GPU. 

    **CPU**  
    - *$\sigma_f$ over Iterations (Left):*  
    This plot displays the evolution of the kernel scale ($\sigma_f$) over iterations. Eventually, the kernel scale converges to ~0.20, which coincidentally is approximately the same as the initial value.   

    - *Length scale over Iterations (Middle):*  
    This plot displays the evolution of the length scale of the kernel over iterations. The length scale converges to ~3.8. Interestingly, the length scale seems to linearly increase during the optimization until it fine tuning is halted. 

    - *$\beta$ over Iterations (Right):*  
    This plot shows the evolution of $\beta$ over iterations. After iteration 150, $\beta$ is reduced with a constant reduction factor of 0.75 per 25 iterations. 
   
   ![CPU 19D Hyperparams](Images/19D_hyperparameter_plot_poster.png)

   Figure 15: Plots showing the evolution of the kernel scale over iterations (left), length scale over iterations (middle), and beta over iterations (right) ran on CPU.
   
   **GPU**

    - *$\sigma_f$ over Iterations (Left):*  
    This plot displays the evolution of the kernel scale ($\sigma_f$) over iterations. Eventually, the kernel scale converges to ~0.33. 

    - *Length scale over Iterations (Middle):*  
    This plot displays the evolution of the length scale of the kernel over iterations. The length scale converges to ~5.0. 

    - *$\beta$ over Iterations (Right):*  
    This plot shows the evolution of $\beta$ over iterations. After iteration 500, $\beta$ is reduced with a constant reduction factor of 0.75 per 25 iterations. 

   ![GPU 19D Hyperparams](Images/19D_hyperparameter_plot_GPU.png)

   Figure 16: Plots showing the evolution of the kernel scale over iterations (left), length scale over iterations (middle), and beta over iterations (right) ran on GPU.

**Conclusion**
 
The 19D optimization reveals that higher dimensional spaces are more challenging for Bayesian optimization, especially if hyperparameter tuning or beta adjustments stop prematurely. Running for more iterations, continuing hyperparameter updates, or employing more advanced dimensionality-reduction strategies could yield better performance. Both CPU and GPU runs demonstrate convergence, but the GPU run’s slightly lower mass suggests that more extensive or longer-running optimization (with continued hyperparameter tuning) can further refine the design in high-dimensional problems.

### Discussion and Conclusion

In this project, the application of Bayesian optimization to a truss structure was demonstrated to minimize structural weight while meeting the natural frequency constraints. In the 1D and 2D cases, the Bayesian optimizer was able to find the optimal solution within a few iterations, showing that this method is extremely effective for low-dimensional optimization problems.

However, a comparison of the results obtained with the Bayesian optimizer and the multi-class teaching-learning-based optimization (MC-TLBO) method from [1], as shown in Table 1, reveals that the lowest weight found by the Bayesian optimizer is significantly higher than the optimal weight achieved by the MC-TLBO. This underscores the challenges faced by the Bayesian optimizer when tackling problems where the dimensionality of the solution space exceeds 10–20 [2]. Even with the use of `1,000` initial points, it was not possible to overcome this limitation. Although the MC-TLBO requires 16,000 iterations to find the optimal solution, we can conclude that this is the more appropriate approach for high-dimensional problems, as it does not pose scalability challenges like the Gaussian process in the Bayesian optimizer. [2]

Table 1: Comparison of the results of the Bayesian optimizer with the Multi-class teaching-learning-based optimization (MC-TLBO) from [1] for the 19D case
|  Bayesian Optimizer | MC-TLBO |
| :-:                 | :-:     |
| ![Sketch_BO](./Images/Truss_19D_BO.jpg) | ![Sketch_TLBO](./Images/Truss_19D_TLBO.jpg)|
| ![Cross sections BO](./Images/Cross_sections_BO.png) | ![Cross sections TLBO](./Images/Cross_sections_TLBO.png) |
| Total weight = 404.95 kg | Total weight = 359.966 kg |
| Number of iterations: 3,000 | Number of iterations: 16,000 |


It is important to mention that these are not the best values obtained, but the best values for 19D. The best obtained parameters are for the GPU run for 6D with a weight of 385.91 kg, as can be seen in the subsection of 6D. 

### References:

[1] Kanarachos, S., Griffin, J., Fitzpatrick, M. E. (2017). Efficient truss optimization using the contrast-based fruit fly optimization algorithm. Computers and Structures, 182, 137-148.

[2] Wang, X., Jin, Y., Schmitt, S., Olhofer, M. (2023). Recent Advances in Bayesian Optimization. ACM Computing Surveys, 55(13), 1-36. https://doi.org/10.1145/3582078

[3] Vink, R. (2019, August 25). Algorithm Breakdown: Bayesian Optimization. Ritchie Vink. https://www.ritchievink.com/blog/2019/08/25/algorithm-breakdown-bayesian-optimization/

 